<a href="https://colab.research.google.com/github/amphyxs/know-graphs-cw/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!pip install -r '/content/requirements.txt'

In [14]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer
from typing import Optional

In [52]:
g = Graph()

g.parse('/content/ontology.rdf')
ONTOLOGY_IRI = 'http://www.semanticweb.org/user/ontologies/2024/9/music-knowledge'

def ref(resource: str) -> URIRef:
  return URIRef(f'{ONTOLOGY_IRI}#{resource}')

In [53]:
class OntologyClass:
  class_name: str
  _class_uri_ref: Optional[URIRef] = None
  uri_ref: URIRef

  @property
  def class_uri_ref(self) -> URIRef:
    if not self._class_uri_ref:
      self._class_uri_ref = ref(self.class_name)

    return self._class_uri_ref

  def __init__(self, name: str, **kwargs: dict) -> None:
    self.uri_ref = ref(name)
    g.add((self.uri_ref, RDF.type, self.class_uri_ref))

In [58]:
# Classes
class Artist(OntologyClass):
  class_name = 'Исполнитель'

class Song(OntologyClass):
  owns_object_property = ref('владеть')
  class_name = 'Композиция'

  def __init__(self, name: str, owner: Artist, **kwargs: dict) -> None:
    super().__init__(name, **kwargs)
    g.add((owner.uri_ref, self.owns_object_property, self.uri_ref))

artist1 = Artist('Queen')
song1 = Song('Bohemian_Rhapsody', owner=artist1)

In [57]:
g.serialize(destination='/content/ontology-from-py.rdf')

<Graph identifier=N20824a8b63e84bcca4b7f6b6e0622964 (<class 'rdflib.graph.Graph'>)>